In [11]:
import numpy as np 
import matplotlib.pyplot as plt
import csv 
import pickle
from scipy import stats
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, f1_score, recall_score
from sklearn.linear_model import LogisticRegression

from domino.data.cxr import get_dp, build_cxr_df, get_cxr_activations, rle2mask


## Extract train/test features from Image-Only model

In [3]:
# Get a mosaic DataPanel with the data.
df = build_cxr_df.out(load=True)
dp = get_dp(df)
dp.head()

,image_id (NumpyArrayColumn),encoded_pixels (NumpyArrayColumn),pmx (NumpyArrayColumn),filepath (NumpyArrayColumn),chest_tube (NumpyArrayColumn),split (NumpyArrayColumn),gaze_seq (NumpyArrayColumn),gaze_heatmap (NumpyArrayColumn),gaze_max_visit (NumpyArrayColumn),gaze_unique (NumpyArrayColumn),gaze_time (NumpyArrayColumn),gaze_diffusivity (NumpyArrayColumn),index (ListColumn),input (CellColumn),img (CellColumn)
0,1.2.276.0.7230010.3.1.4.8323329.6904.151787520...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,'0',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
1,1.2.276.0.7230010.3.1.4.8323329.13666.15178752...,557374 2 1015 8 1009 14 1002 20 997 26 990 32 ...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,'1',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
2,1.2.276.0.7230010.3.1.4.8323329.11028.15178752...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,'2',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
3,1.2.276.0.7230010.3.1.4.8323329.10366.15178752...,514175 10 1008 29 994 30 993 32 991 33 990 34 ...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,'3',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
4,1.2.276.0.7230010.3.1.4.8323329.10016.15178752...,592184 33 976 58 956 73 941 88 926 102 917 109...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,'4',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...


In [4]:
model_pth = "/media/nvme_data/observational_results_10_2020/original/cxr/emmental_cam/cam_0/seed_0/best_model_target_cxr_val_accuracy.pth"

act_dp = get_cxr_activations(dp=dp, model_path=model_pth)


In [5]:
# train mask are points that have gaze
train_mask = dp["gaze_seq"].data != "nan"
act_dp[train_mask].head()

,image_id (NumpyArrayColumn),encoded_pixels (NumpyArrayColumn),pmx (NumpyArrayColumn),filepath (NumpyArrayColumn),chest_tube (NumpyArrayColumn),split (NumpyArrayColumn),gaze_seq (NumpyArrayColumn),gaze_heatmap (NumpyArrayColumn),gaze_max_visit (NumpyArrayColumn),gaze_unique (NumpyArrayColumn),gaze_time (NumpyArrayColumn),gaze_diffusivity (NumpyArrayColumn),index (ListColumn),input (TensorColumn),img (ListColumn),pred (NumpyArrayColumn),probs (NumpyArrayColumn),activation_block4 (NumpyArrayColumn)
0,1.2.276.0.7230010.3.1.4.8323329.1857.151787516...,378999 2 1018 8 1013 13 1009 15 1007 17 1006 1...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,1.0,train,"[(0.3002531741212215, 0.9348943074817682, 2), ...",[[ 0. 0. 0. 14. 2. 2. 1. 0.]\n [ 0. 0. ...,14.0,12.0,33.0,0.545455,'29',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
1,1.2.276.0.7230010.3.1.4.8323329.1219.151787516...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,1.0,train,"[(0.3135288701615107, 0.5596042556980056, 2), ...",[[0. 0. 0. 1. 3. 0. 0. 0.]\n [0. 0. 0. 3. 3. 2...,7.0,19.0,46.0,0.369565,'39',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
2,1.2.276.0.7230010.3.1.4.8323329.518.1517875163...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,"[(0.12663856335055726, 0.6144331769405472, 2),...",[[ 0. 0. 0. 0. 0. 0. 0. 0.]\n [ 0. 0. ...,12.0,24.0,77.0,0.376623,'40',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
3,1.2.276.0.7230010.3.1.4.8323329.2118.151787517...,591104 8 1011 18 1001 26 993 32 988 36 986 37 ...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,1.0,train,"[(0.1227472372112641, 0.5808183519907184, 3), ...",[[ 0. 0. 0. 0. 4. 1. 0. 1.]\n [ 0. 0. ...,10.0,23.0,81.0,0.395062,'43',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
4,1.2.276.0.7230010.3.1.4.8323329.2027.151787517...,500845 25 992 38 983 41 980 44 978 46 975 49 9...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,0.0,train,"[(0.3294461311053661, 0.5748967530438639, 4), ...",[[ 0. 0. 4. 1. 10. 8. 0. 0.]\n [ 0. 0. ...,11.0,21.0,78.0,0.641026,'50',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"


In [6]:
# test mask are points that do not have gaze and have chest tube labels
test_mask = np.logical_and(~train_mask, ~np.isnan(dp["chest_tube"]))
act_dp[test_mask].head()

,image_id (NumpyArrayColumn),encoded_pixels (NumpyArrayColumn),pmx (NumpyArrayColumn),filepath (NumpyArrayColumn),chest_tube (NumpyArrayColumn),split (NumpyArrayColumn),gaze_seq (NumpyArrayColumn),gaze_heatmap (NumpyArrayColumn),gaze_max_visit (NumpyArrayColumn),gaze_unique (NumpyArrayColumn),gaze_time (NumpyArrayColumn),gaze_diffusivity (NumpyArrayColumn),index (ListColumn),input (TensorColumn),img (ListColumn),pred (NumpyArrayColumn),probs (NumpyArrayColumn),activation_block4 (NumpyArrayColumn)
0,1.2.276.0.7230010.3.1.4.8323329.32395.15178751...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,0.0,test,nan,nan,NaN,NaN,NaN,NaN,'7',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
1,1.2.276.0.7230010.3.1.4.8323329.12084.15178752...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,0.0,test,nan,nan,NaN,NaN,NaN,NaN,'26',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
2,1.2.276.0.7230010.3.1.4.8323329.13325.15178752...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,0.0,test,nan,nan,NaN,NaN,NaN,NaN,'36',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
3,1.2.276.0.7230010.3.1.4.8323329.11640.15178752...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,0.0,test,nan,nan,NaN,NaN,NaN,NaN,'47',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"
4,1.2.276.0.7230010.3.1.4.8323329.14529.15178752...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,0.0,test,nan,nan,NaN,NaN,NaN,NaN,'66',"np.ndarray(shape=torch.Size([3, 224, 224]))",<PIL.Image.Image image mode=L size=1024x1024 a...,0,"np.ndarray(shape=(2,))","np.ndarray(shape=(2048, 7, 7))"


In [7]:
# act_dp_ = get_cxr_activations.out(run_id=35,load=True)
# act_dp_.head()

In [8]:
image_feats_train = act_dp["activation_block4"][train_mask].reshape(-1,2048,49).mean(2)
tube_labels_train = act_dp["chest_tube"][train_mask]
pmx_labels_train = act_dp["pmx"][train_mask]
probs_train = act_dp["probs"][train_mask].data[:,1]
gaze_feature = act_dp["gaze_time"][train_mask]

image_feats_test = act_dp[test_mask]["activation_block4"].reshape(-1,2048,49).mean(2)
tube_labels_test = act_dp["chest_tube"][test_mask]
pmx_labels_test = act_dp["pmx"][test_mask]
probs_test = act_dp["probs"][test_mask].data[:,1]

## train few-shot LR models with gaze features

In [9]:
k = 20
label_mask = pmx_labels_train == 1
label_mask_test = pmx_labels_test > -1


thresh1 = np.sort(gaze_feature[label_mask])[-k]
thresh2 = np.sort(gaze_feature[label_mask])[k]
label_mask_1 = gaze_feature[label_mask] >= thresh1
label_mask_0 = gaze_feature[label_mask] < thresh2

X_train = np.concatenate((image_feats_train[label_mask][label_mask_1],image_feats_train[label_mask][label_mask_0]))
y_train = np.array([1]*label_mask_1.sum() + [0]*label_mask_0.sum())

print(X_train.shape)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
test_slices = clf.predict(image_feats_test[label_mask_test])
print(f"{test_slices.sum()}/{len(test_slices)}")


(39, 2048)
366/1000


## View performance gaps in test set

In [16]:
thresh = 0.3 #373 # this threshold achieves 0.55 recall for overall
preds_test = (probs_test > thresh)
overall_acc = (preds_test[label_mask_test] == pmx_labels_test[label_mask_test]).mean()
#print(f"Overall accuracy for class: {100*overall_acc:.2f}")
print(f"Overall AUROC: {100*roc_auc_score(pmx_labels_test, probs_test):.2f}")


mask1 = test_slices.astype(bool)
acc_0 = (preds_test[label_mask_test][mask1] == pmx_labels_test[label_mask_test][mask1]).mean()
#print(f"Accuracy for class on slice 0: {100*acc_0:.2f}")
print(f"Mask1 AUROC: {100*roc_auc_score(pmx_labels_test[mask1], probs_test[mask1]):.2f}")


mask2 = (1-test_slices).astype(bool)
acc_1 = (preds_test[label_mask_test][mask2] == pmx_labels_test[label_mask_test][mask2]).mean()
#print(f"Accuracy for class on slice 1: {100*acc_1:.2f}")
print(f"Mask2 AUROC: {100*roc_auc_score(pmx_labels_test[mask2], probs_test[mask2]):.2f}")


Overall AUROC: 90.86
Mask1 AUROC: 82.88
Mask2 AUROC: 92.32


## Quantitative analysis with segmentation masks

In [35]:
rle_1 = act_dp[test_mask]["encoded_pixels"][mask1]
rle_2 = act_dp[test_mask]["encoded_pixels"][mask2]

In [36]:
segmasks_1 = np.array([rle2mask(rle,1024,1024) for rle in rle_1 if rle!='-1'])
segmasks_2 = np.array([rle2mask(rle,1024,1024) for rle in rle_2 if rle!='-1'])


In [39]:
abn_size1 = segmasks_1.reshape(-1,1024*1024).mean(1)
abn_size2 = segmasks_2.reshape(-1,1024*1024).mean(1)

print(f"Average abnormal area for mask1: {abn_size1.mean():.4f}")
print(f"Average abnormal area for mask2: {abn_size2.mean():.4f}")

from scipy.stats import ttest_ind

print(f"p-value: {ttest_ind(abn_size1,abn_size2)[1]:.4f}")


Average abnormal area for mask1: 0.0098
Average abnormal area for mask2: 0.0146
p-value: 0.0115
